In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline
import sys
import os
import math
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Circle
import seaborn as sns; 
from IPython.display import HTML

In [ ]:
#config parser
import configparser

sys.path.insert(0, '../Common/')
from AISDataManager import AISDataManager
import Constants as c
import HMUtils as hMUtil
import TimeUtils as timeUtils
import GeoCompute as gC

#MyConfig.INI stores all the run time constants
config = configparser.ConfigParser()
config.read('../MyConfig.INI')

from joblib import Parallel, delayed
import multiprocessing
aISDM = AISDataManager()

In [ ]:
lonMin = (float)(config['TRAJ_PRED_LSTM_TYPE_LEN_TS']['LON_MIN'])
lonMax = (float)(config['TRAJ_PRED_LSTM_TYPE_LEN_TS']['LON_MAX'])

latMin = (float)(config['TRAJ_PRED_LSTM_TYPE_LEN_TS']['LAT_MIN'])
latMax = (float)(config['TRAJ_PRED_LSTM_TYPE_LEN_TS']['LAT_MAX'])

print(lonMin,latMin)
print(lonMax,latMax)

increStep = (float)(config['TRAJ_PRED_LSTM_TYPE_LEN_TS']['INCR_STEP'])
incrRes = (int)(config['TRAJ_PRED_LSTM_TYPE_LEN_TS']['INCR_RES'])

sourceDir1 = config['TRAJ_PRED_LSTM_TYPE_LEN_TS']['SOURCE_DIR_1']
sourceDir2 = config['TRAJ_PRED_LSTM_TYPE_LEN_TS']['SOURCE_DIR_2']

trainTrajNum1 = (int)(config['TRAJ_PRED_LSTM_TYPE_LEN_TS']['TRAIN_DATA_1'])
trainTrajNum2 = (int)(config['TRAJ_PRED_LSTM_TYPE_LEN_TS']['TRAIN_DATA_2'])

testEndTrajNum1 = (int)(config['TRAJ_PRED_LSTM_TYPE_LEN_TS']['TEST_END_1'])
testEndTrajNum2 = (int)(config['TRAJ_PRED_LSTM_TYPE_LEN_TS']['TEST_END_2'])

dataDir = config['TRAJ_PRED_LSTM_TYPE_LEN_TS']['DATA_DIR']
    
print(sourceDir1)
print(sourceDir2)
print(trainTrajNum1, testEndTrajNum1)
print(trainTrajNum2, testEndTrajNum2)

print(dataDir)

In [ ]:
class VesselTypeSource:
    """
    """
    def __init__(self, srcDir, trainNum, testNumEnd):
        self.srcDir = srcDir
        self.trainNum = trainNum
        self.testNumEnd = testNumEnd
        
    def __str__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

cargo1004 = VesselTypeSource(sourceDir1, trainTrajNum1, testEndTrajNum1)
tanker1024 = VesselTypeSource(sourceDir2, trainTrajNum2, testEndTrajNum2)

vesselDataSources = [cargo1004, tanker1024]
print(vesselDataSources[0])
print(vesselDataSources[1])

In [ ]:
heatMapGrid = hMUtil.generate_grid(lonMin, lonMax, latMin, latMax, increStep, incrRes)
boundaryArray = heatMapGrid[2]
horizontalAxis = heatMapGrid[0]
verticalAxis = heatMapGrid[1]
totalStates = horizontalAxis.shape[0] * verticalAxis.shape[0]

In [ ]:
def get_traj_lon_lat_data(sourceDir, num):
    #read the dataframe
    sorceFile = sourceDir + str(num) + '.csv'
    sourceDF,_ = aISDM.load_data_from_csv(sorceFile)
    return sourceDF.loc[:,['LON','LAT']].to_numpy()
    
get_traj_lon_lat_data(vesselDataSources[0].srcDir, 0)

In [ ]:
def get_traj_lon_lat_data_with_len(sourceDir, num):
    sorceFile = sourceDir + str(num) + '.csv'
    sourceDF,_ = aISDM.load_data_from_csv(sorceFile)
    return sourceDF.loc[:,['LON','LAT']].to_numpy(), sourceDF.loc[0,'Length']

get_traj_lon_lat_data_with_len(vesselDataSources[0].srcDir, 0)

In [ ]:
#make list of all such trajectories
#this is training data
trajSeqList = []
typeList = []
diffVessel = 0
for vesselDataSource in vesselDataSources:
    print("Taking data from:%s"%vesselDataSource.srcDir)
    for trajNum in range(0,vesselDataSource.trainNum):
        trajSeqList.append(get_traj_lon_lat_data_with_len(vesselDataSource.srcDir,trajNum))
        #append the type information as well
        typeList.append(diffVessel)
    diffVessel = diffVessel + 1

In [ ]:
print(trajSeqList[0])

In [ ]:
def convert_seq_to_x_y_lon_lat(seq, typeVessel, lenShip):
    #first column
    #-2 is is to take care of boundary condition
    #since we are considering 2 time stamps for the input data
    firstTwoCol = seq[:-2,:].copy()
    secTwoCol = seq[1:-1,:].copy()

    outputLabel = seq[2:,:].copy()
    
    xDataRow = firstTwoCol.shape[0]
    typeData = np.zeros((xDataRow,2))
    #either 0 or 1
    typeData[:,typeVessel] = 1
    
    lenData = np.zeros((xDataRow,1))
    lenData[:,:] = lenShip
    #FIXME make this more generic
    xData = np.hstack((firstTwoCol,typeData,lenData,secTwoCol,typeData,lenData))
    return xData, outputLabel
convert_seq_to_x_y_lon_lat(trajSeqList[0][0],typeList[0],trajSeqList[0][1])

In [ ]:
#now iterate throgh trajSeqList 
#and keep on stacking them vertically
#to make giant input and output matrix
xData = np.zeros((0,10))
yData = np.zeros((0,2))
print(xData.shape)
print(yData.shape)
for trajNum in range(len(trajSeqList)):
    if((trajSeqList[trajNum][0].shape[0]) > 2):
        xTemp,yTemp = convert_seq_to_x_y_lon_lat(trajSeqList[trajNum][0],typeList[trajNum],trajSeqList[trajNum][1])
        xData = np.vstack((xData,xTemp.copy()))
        yData = np.vstack((yData,yTemp.copy()))
        
print(xData)
print(yData)

In [ ]:
print(xData.shape)
print(yData.shape)

In [ ]:
xToStore = dataDir + "XData.npy"
yToStore = dataDir + "YData.npy"
np.save(xToStore, xData)
np.save(yToStore, yData)

In [ ]:
xToStore = dataDir + "XData.npy"
yToStore = dataDir + "YData.npy"

xData = np.load(xToStore)
yData = np.load(yToStore)

In [ ]:
print(xData.shape)
print(yData.shape)

In [ ]:
#to get the min and max stats of len
lenListNP = xData[:,-1]
lenListNPMin = np.min(lenListNP)
lenListNPMax = np.max(lenListNP)
print(lenListNPMin)
print(lenListNPMax)

In [ ]:
#normalise the data
xLonLatData_0 = (xData[:,0] - lonMin)/(lonMax - lonMin)
xLonLatData_0 = np.reshape(xLonLatData_0,(xLonLatData_0.shape[0],1))
xLonLatData_1 = (xData[:,1] - latMin)/(latMax - latMin)
xLonLatData_1 = np.reshape(xLonLatData_1,(xLonLatData_1.shape[0],1))

xLonLatData_2 = xData[:,2].copy()
xLonLatData_2 = np.reshape(xLonLatData_2,(xLonLatData_2.shape[0],1))
xLonLatData_3 = xData[:,3].copy()
xLonLatData_3 = np.reshape(xLonLatData_3,(xLonLatData_3.shape[0],1))

xLonLatData_4 = (xData[:,4] - lenListNPMin)/(lenListNPMax - lenListNPMin)
xLonLatData_4 = np.reshape(xLonLatData_4,(xLonLatData_4.shape[0],1))

xLonLatData_5 = (xData[:,5] - lonMin)/(lonMax - lonMin)
xLonLatData_5 = np.reshape(xLonLatData_5,(xLonLatData_5.shape[0],1))
xLonLatData_6 = (xData[:,6] - latMin)/(latMax - latMin)
xLonLatData_6 = np.reshape(xLonLatData_6,(xLonLatData_6.shape[0],1))

xLonLatData_7 = xData[:,7].copy()
xLonLatData_7 = np.reshape(xLonLatData_7,(xLonLatData_7.shape[0],1))
xLonLatData_8 = xData[:,8].copy()
xLonLatData_8 = np.reshape(xLonLatData_8,(xLonLatData_8.shape[0],1))

xLonLatData_9 = (xData[:,9] - lenListNPMin)/(lenListNPMax - lenListNPMin)
xLonLatData_9 = np.reshape(xLonLatData_9,(xLonLatData_9.shape[0],1))

xLonLatDataNorm = np.hstack((xLonLatData_0,xLonLatData_1  \
                             ,xLonLatData_2,xLonLatData_3 \
                             ,xLonLatData_4,xLonLatData_5 \
                             ,xLonLatData_6,xLonLatData_7 \
                             ,xLonLatData_8,xLonLatData_9 \
                            ))

xLonLatDataNorm = np.reshape(xLonLatDataNorm,(xLonLatDataNorm.shape[0], 2, 5))

In [ ]:
print(xLonLatDataNorm.shape)

In [ ]:
#normalise the output data as well
yLonLatData_0 = (yData[:,0] - lonMin)/(lonMax - lonMin)
yLonLatData_0 = np.reshape(yLonLatData_0,(yLonLatData_0.shape[0],1))
yLonLatData_1 = (yData[:,1] - latMin)/(latMax - latMin)
yLonLatData_1 = np.reshape(yLonLatData_1,(yLonLatData_1.shape[0],1))
yLonLatDataNorm = np.hstack((yLonLatData_0,yLonLatData_1))

In [ ]:
print(yLonLatDataNorm.shape)

In [ ]:
print(xLonLatDataNorm[0,:])
print(yLonLatDataNorm[0,:])

In [ ]:
from keras import Sequential
from keras.layers import Dense, LSTM
from keras.models import Model
from keras.layers import Input
from keras.layers import concatenate

In [ ]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [ ]:
model = Sequential()
model.add(LSTM(units=50, return_sequences= True, input_shape=(2,5)))
model.add(LSTM(units=50))
model.add(Dense(150, activation='relu'))
model.add(Dense(units=2, activation='linear'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.fit(xLonLatDataNorm, yLonLatDataNorm, epochs=500, batch_size = 512 , verbose = 2)
# model.fit([xLonLatTSNorm, xTypeLenNorm], yLonLatDataNorm, epochs=200, batch_size = 512 , verbose = 1)

In [ ]:
modelDir = dataDir + "Model_500_MSE_2.h5"

In [ ]:
model.save(modelDir)

In [ ]:
from keras.models import load_model
model = load_model(modelDir)

In [ ]:
#will take four arguments
#[[lonPrev, latPrev, lonCurr, latCurr]] shape is 1x4 or nx4
#can be vectorised also
def normalize_lon_lat(arr):
    #subtract the minimum 
    #and divide by range
    ret0 = (arr[:,0] - lonMin)/(lonMax - lonMin)
    ret0 = np.reshape(ret0, (ret0.shape[0],1))
    ret1 = (arr[:,1] - latMin)/(latMax - latMin)
    ret1 = np.reshape(ret1, (ret1.shape[0],1))
    ret = np.hstack((ret0, ret1))
    return ret

In [ ]:
#takes 2 time stamps of LON and LAT
#normalises them and 
#and makes the prediction
#de normalize the output
#and return the values
#prevTraj numpy array of 1x2
#currTraj numpy array of 1x2
def compute_30_min_pred(prevTraj, currTraj, typeVessel, lenShip):
    """
    Compute prediction for 30 minutes.
    
    takes LON and LAT of three time stamps
    normalises them
    and makes prediction
    and returns denormalised LON and LAT values
    """
        
    if(prevTraj.shape != (1,2)):
        raise Exception('Shape of prevTraj should be 1x2')
    
    if(currTraj.shape != (1,2)):
        raise Exception('Shape of currTraj should be 1x2')
        
    trajX = np.vstack((prevTraj,currTraj))
    trajXNorm = normalize_lon_lat(trajX)
    
    
    typeData = np.zeros((2,2))
    #either 0 or 1
    typeData[:,typeVessel] = 1
    
    lenData = np.zeros((2,1))
    lenData[:,:] = lenShip
    
    lenData = (lenData - lenListNPMin)/(lenListNPMax - lenListNPMin)
    
    trajXNormData = np.hstack((trajXNorm,typeData,lenData))
    
    trajXNormData = np.reshape(trajXNormData,(1,2,5))
    predLatLon = model.predict(trajXNormData)
    
    predLon = predLatLon[0,0]
    predLat = predLatLon[0,1]
    
    #after prediction de normalise it
    predLonScaled = (predLon * (lonMax - lonMin)) + lonMin
    predLatScaled = (predLat * (latMax - latMin)) + latMin
    return predLonScaled, predLatScaled

In [ ]:
def compute_n_30_min_pred(prevTraj, currTraj, typeVessel, lenShip, n = 1):
    """
    Compute prediction for n*30 minutes.
    
    calls compute_30_min_pred
    according to value of n
    default value of n = 1
    """
    #temp vaariables to store the previous trajectory 
    firstLoc = prevTraj.copy()
    secLoc = currTraj.copy()
    
    #return value will be numpy array of nx2
    ret = np.zeros((0,2))
    
    for i in range(n):
        #make prediction using 30 min pred
        predLon, predLat = compute_30_min_pred(firstLoc, secLoc, typeVessel, lenShip)
        #append them in ret
        ret = np.vstack((ret,np.array([[predLon,predLat]])))
        
        #update firstLoc and secLoc 
        #for next  iteration
        
        firstLoc = secLoc.copy()
        secLoc = np.reshape(ret[-1,:].copy(),(1,2))
    return ret

In [ ]:
def get_error_for_traj(srcDir, num, typeVessel):
    """
    Compute error for the entire trajectory.
    
    Returns list of error for predictions of upto 
    4 hours i.e. 240 minutes.
    
    Parameters:
        num (int): number of vessel trajectory 
                   whose prediction errors to be computed.
    Returns:
        errorVal (list of floats): list of errors in KM for each
                                   30 min predictions.
    """    
    errorVal = []
    vesselTraj,lenData = get_traj_lon_lat_data_with_len(srcDir, num)

    if(vesselTraj.shape[0] < 3):
        return errorVal
        
        
    predVesselTraj = compute_n_30_min_pred(np.reshape(vesselTraj[0,:], (1,2)), np.reshape(vesselTraj[1,:], (1,2)), typeVessel, lenData, n = 8)
    predRange = vesselTraj.shape[0] - 2

    #FIXME 8 can come from some variable too
    #for 8 consecutive predictions
    if(predRange > 8):
        predRange = 8

    #use trajectories which has atleast one
    #value to predict
    if(predRange >= 0):
        for pred in range(predRange):    
            trueLon, trueLat = vesselTraj[(2 + pred),0], vesselTraj[(2 + pred),1]
            predLon, predLat = predVesselTraj[pred,0], predVesselTraj[pred,1]
    #         print(trueLon, trueLat, '-', predLon, predLat)
            errorVal.append(gC.compute_distance(trueLon, trueLat, predLon, predLat))
    return errorVal

In [ ]:
help(get_error_for_traj)

In [ ]:
#store errors for all training trajectories
trainDataWholeErrors_1004 = []
for traj in range(vesselDataSources[0].trainNum):
    trainDataWholeErrors_1004.append(get_error_for_traj(vesselDataSources[0].srcDir,traj,0))

In [ ]:
#segregate those list of errors into
#list of 30 minErr, 60 minErr, ...
trainWholeErr_1004_30 = []
trainWholeErr_1004_60 = []
trainWholeErr_1004_90 = []
trainWholeErr_1004_120 = []
trainWholeErr_1004_150 = []
trainWholeErr_1004_180 = []
trainWholeErr_1004_210 = []
trainWholeErr_1004_240 = []

trainWholeErr_1004_n_30 = [trainWholeErr_1004_30 \
, trainWholeErr_1004_60 \
, trainWholeErr_1004_90 \
, trainWholeErr_1004_120 \
, trainWholeErr_1004_150 \
, trainWholeErr_1004_180 \
, trainWholeErr_1004_210 \
, trainWholeErr_1004_240 \
]

for trajErr in trainDataWholeErrors_1004:
    #take the list
    for n_30 in range(8):
        if(len(trajErr) > n_30):
            trainWholeErr_1004_n_30[n_30].append(trajErr[n_30])

In [ ]:
def format_func(value, tick_number):
    tempTick = (value*30) + 30
    ret = "%d"%(tempTick)
    return ret

In [ ]:
trainWholeErr_1004_30NP = np.array(trainWholeErr_1004_30)
trainWholeErr_1004_60NP = np.array(trainWholeErr_1004_60)
trainWholeErr_1004_90NP = np.array(trainWholeErr_1004_90)
trainWholeErr_1004_120NP = np.array(trainWholeErr_1004_120)
trainWholeErr_1004_150NP = np.array(trainWholeErr_1004_150)
trainWholeErr_1004_180NP = np.array(trainWholeErr_1004_180)
trainWholeErr_1004_210NP = np.array(trainWholeErr_1004_210)
trainWholeErr_1004_240NP = np.array(trainWholeErr_1004_240)

In [ ]:
trainWholeErrMean_1004 = [np.mean(trainWholeErr_1004_30NP) \
            ,np.mean(trainWholeErr_1004_60NP) \
            ,np.mean(trainWholeErr_1004_90NP) \
            ,np.mean(trainWholeErr_1004_120NP) \
            ,np.mean(trainWholeErr_1004_150NP) \
            ,np.mean(trainWholeErr_1004_180NP) \
            ,np.mean(trainWholeErr_1004_210NP) \
            ,np.mean(trainWholeErr_1004_240NP) \
           ]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
ax.set_title("Average Error Value On Training 1004")
ax.set_ylabel("Distance in KM")
ax.set_xlabel("Time in Minutes")
ax.plot(trainWholeErrMean_1004,label = "LSTM")
ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
ax.legend()

In [ ]:
#store errors for all training trajectories
testDataWholeErrors_1004 = []
for traj in range(vesselDataSources[0].trainNum,vesselDataSources[0].testNumEnd):
    testDataWholeErrors_1004.append(get_error_for_traj(vesselDataSources[0].srcDir,traj,0))

In [ ]:
#segregate those list of errors into
#list of 30 minErr, 60 minErr, ...
testWholeErr_1004_30 = []
testWholeErr_1004_60 = []
testWholeErr_1004_90 = []
testWholeErr_1004_120 = []
testWholeErr_1004_150 = []
testWholeErr_1004_180 = []
testWholeErr_1004_210 = []
testWholeErr_1004_240 = []

testWholeErr_1004_n_30 = [testWholeErr_1004_30 \
, testWholeErr_1004_60 \
, testWholeErr_1004_90 \
, testWholeErr_1004_120 \
, testWholeErr_1004_150 \
, testWholeErr_1004_180 \
, testWholeErr_1004_210 \
, testWholeErr_1004_240 \
]

for trajErr in testDataWholeErrors_1004:
    #take the list
    for n_30 in range(8):
        if(len(trajErr) > n_30):
            testWholeErr_1004_n_30[n_30].append(trajErr[n_30])

In [ ]:
testWholeErr_1004_30NP = np.array(testWholeErr_1004_30)
testWholeErr_1004_60NP = np.array(testWholeErr_1004_60)
testWholeErr_1004_90NP = np.array(testWholeErr_1004_90)
testWholeErr_1004_120NP = np.array(testWholeErr_1004_120)
testWholeErr_1004_150NP = np.array(testWholeErr_1004_150)
testWholeErr_1004_180NP = np.array(testWholeErr_1004_180)
testWholeErr_1004_210NP = np.array(testWholeErr_1004_210)
testWholeErr_1004_240NP = np.array(testWholeErr_1004_240)

In [ ]:
testWholeErrMean_1004 = [np.mean(testWholeErr_1004_30NP) \
            ,np.mean(testWholeErr_1004_60NP) \
            ,np.mean(testWholeErr_1004_90NP) \
            ,np.mean(testWholeErr_1004_120NP) \
            ,np.mean(testWholeErr_1004_150NP) \
            ,np.mean(testWholeErr_1004_180NP) \
            ,np.mean(testWholeErr_1004_210NP) \
            ,np.mean(testWholeErr_1004_240NP) \
           ]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
ax.set_title("Average Error Value On Testing 1004")
ax.set_ylabel("Distance in KM")
ax.set_xlabel("Time in Minutes")
ax.plot(testWholeErrMean_1004,label = "LSTM")
ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
ax.legend()

In [ ]:
#store errors for all training trajectories
trainDataWholeErrors_1024 = []
for traj in range(vesselDataSources[1].trainNum):
    trainDataWholeErrors_1024.append(get_error_for_traj(vesselDataSources[1].srcDir,traj,1))

In [ ]:
#store errors for all training trajectories
testDataWholeErrors_1024 = []
for traj in range(vesselDataSources[1].trainNum,vesselDataSources[1].testNumEnd):
    testDataWholeErrors_1024.append(get_error_for_traj(vesselDataSources[1].srcDir,traj,1))

In [ ]:
np.random.seed(0)
rand1000TrajTrain = np.random.randint(0,vesselDataSources[0].trainNum,size = 1000).tolist()

In [ ]:
trainR1000Err = []
for traj in rand1000TrajTrain:
    trainR1000Err.append(get_error_for_traj(vesselDataSources[0].srcDir,traj,0))

In [ ]:
#segregate those list of errors into
#list of 30 minErr, 60 minErr, ...
trainR1000Err_30 = []
trainR1000Err_60 = []
trainR1000Err_90 = []
trainR1000Err_120 = []
trainR1000Err_150 = []
trainR1000Err_180 = []
trainR1000Err_210 = []
trainR1000Err_240 = []

trainR1000Err_n_30 = [trainR1000Err_30 \
, trainR1000Err_60 \
, trainR1000Err_90 \
, trainR1000Err_120 \
, trainR1000Err_150 \
, trainR1000Err_180 \
, trainR1000Err_210 \
, trainR1000Err_240 \
]

for trajErr in trainR1000Err:
    #take the list
    for n_30 in range(8):
        if(len(trajErr) > n_30):
            trainR1000Err_n_30[n_30].append(trajErr[n_30])

In [ ]:
trainR1000Err_30NP = np.array(trainR1000Err_30)
trainR1000Err_60NP = np.array(trainR1000Err_60)
trainR1000Err_90NP = np.array(trainR1000Err_90)
trainR1000Err_120NP = np.array(trainR1000Err_120)
trainR1000Err_150NP = np.array(trainR1000Err_150)
trainR1000Err_180NP = np.array(trainR1000Err_180)
trainR1000Err_210NP = np.array(trainR1000Err_210)
trainR1000Err_240NP = np.array(trainR1000Err_240)

In [ ]:
trainR1000ErrMean = [np.mean(trainR1000Err_30NP) \
            ,np.mean(trainR1000Err_60NP) \
            ,np.mean(trainR1000Err_90NP) \
            ,np.mean(trainR1000Err_120NP) \
            ,np.mean(trainR1000Err_150NP) \
            ,np.mean(trainR1000Err_180NP) \
            ,np.mean(trainR1000Err_210NP) \
            ,np.mean(trainR1000Err_240NP) \
           ]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
ax.set_title("Average Error for 1000 Training Trajectories")
ax.set_ylabel("Distance in KM")
ax.set_xlabel("Time in Minutes")
ax.plot(trainR1000ErrMean,label = "LSTM")
ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
ax.legend()

In [ ]:
np.random.seed(1)
rand1000TrajTest = np.random.randint(vesselDataSources[0].trainNum,vesselDataSources[0].testNumEnd,size = 1000).tolist()

In [ ]:
testR1000Err = []
for traj in rand1000TrajTrain:
    testR1000Err.append(get_error_for_traj(vesselDataSources[0].srcDir,traj,0))

In [ ]:
#segregate those list of errors into
#list of 30 minErr, 60 minErr, ...
testR1000Err_30 = []
testR1000Err_60 = []
testR1000Err_90 = []
testR1000Err_120 = []
testR1000Err_150 = []
testR1000Err_180 = []
testR1000Err_210 = []
testR1000Err_240 = []

testR1000Err_n_30 = [testR1000Err_30 \
, testR1000Err_60 \
, testR1000Err_90 \
, testR1000Err_120 \
, testR1000Err_150 \
, testR1000Err_180 \
, testR1000Err_210 \
, testR1000Err_240 \
]

for trajErr in testR1000Err:
    #take the list
    for n_30 in range(8):
        if(len(trajErr) > n_30):
            testR1000Err_n_30[n_30].append(trajErr[n_30])

In [ ]:
testR1000Err_30NP = np.array(testR1000Err_30)
testR1000Err_60NP = np.array(testR1000Err_60)
testR1000Err_90NP = np.array(testR1000Err_90)
testR1000Err_120NP = np.array(testR1000Err_120)
testR1000Err_150NP = np.array(testR1000Err_150)
testR1000Err_180NP = np.array(testR1000Err_180)
testR1000Err_210NP = np.array(testR1000Err_210)
testR1000Err_240NP = np.array(testR1000Err_240)

In [ ]:
testR1000ErrMean = [np.mean(testR1000Err_30NP) \
            ,np.mean(testR1000Err_60NP) \
            ,np.mean(testR1000Err_90NP) \
            ,np.mean(testR1000Err_120NP) \
            ,np.mean(testR1000Err_150NP) \
            ,np.mean(testR1000Err_180NP) \
            ,np.mean(testR1000Err_210NP) \
            ,np.mean(testR1000Err_240NP) \
           ]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
ax.set_title("Average Error for 1000 Testing Trajectories")
ax.set_ylabel("Distance in KM")
ax.set_xlabel("Time in Minutes")
ax.plot(testR1000ErrMean,label = "LSTM")
ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
ax.legend()

In [ ]:
dataToStore = dataDir + "trainWholeErr_1004_30NP.npy"
np.save(dataToStore, trainWholeErr_1004_30NP)
dataToStore = dataDir + "trainWholeErr_1004_60NP.npy"
np.save(dataToStore, trainWholeErr_1004_60NP)
dataToStore = dataDir + "trainWholeErr_1004_90NP.npy"
np.save(dataToStore, trainWholeErr_1004_90NP)
dataToStore = dataDir + "trainWholeErr_1004_120NP.npy"
np.save(dataToStore, trainWholeErr_1004_120NP)
dataToStore = dataDir + "trainWholeErr_1004_150NP.npy"
np.save(dataToStore, trainWholeErr_1004_150NP)
dataToStore = dataDir + "trainWholeErr_1004_180NP.npy"
np.save(dataToStore, trainWholeErr_1004_180NP)
dataToStore = dataDir + "trainWholeErr_1004_210NP.npy"
np.save(dataToStore, trainWholeErr_1004_210NP)
dataToStore = dataDir + "trainWholeErr_1004_240NP.npy"
np.save(dataToStore, trainWholeErr_1004_240NP)

In [ ]:
dataToStore = dataDir + "testWholeErr_1004_30NP.npy"
np.save(dataToStore, testWholeErr_1004_30NP)
dataToStore = dataDir + "testWholeErr_1004_60NP.npy"
np.save(dataToStore, testWholeErr_1004_60NP)
dataToStore = dataDir + "testWholeErr_1004_90NP.npy"
np.save(dataToStore, testWholeErr_1004_90NP)
dataToStore = dataDir + "testWholeErr_1004_120NP.npy"
np.save(dataToStore, testWholeErr_1004_120NP)
dataToStore = dataDir + "testWholeErr_1004_150NP.npy"
np.save(dataToStore, testWholeErr_1004_150NP)
dataToStore = dataDir + "testWholeErr_1004_180NP.npy"
np.save(dataToStore, testWholeErr_1004_180NP)
dataToStore = dataDir + "testWholeErr_1004_210NP.npy"
np.save(dataToStore, testWholeErr_1004_210NP)
dataToStore = dataDir + "testWholeErr_1004_240NP.npy"
np.save(dataToStore, testWholeErr_1004_240NP)

In [ ]:
dataToStore = dataDir + "trainR1000Err_30NP.npy"
np.save(dataToStore, trainR1000Err_30NP)
dataToStore = dataDir + "trainR1000Err_60NP.npy"
np.save(dataToStore, trainR1000Err_60NP)
dataToStore = dataDir + "trainR1000Err_90NP.npy"
np.save(dataToStore, trainR1000Err_90NP)
dataToStore = dataDir + "trainR1000Err_120NP.npy"
np.save(dataToStore, trainR1000Err_120NP)
dataToStore = dataDir + "trainR1000Err_150NP.npy"
np.save(dataToStore, trainR1000Err_150NP)
dataToStore = dataDir + "trainR1000Err_180NP.npy"
np.save(dataToStore, trainR1000Err_180NP)
dataToStore = dataDir + "trainR1000Err_210NP.npy"
np.save(dataToStore, trainR1000Err_210NP)
dataToStore = dataDir + "trainR1000Err_240NP.npy"
np.save(dataToStore, trainR1000Err_240NP)

In [ ]:
dataToStore = dataDir + "testR1000Err_30NP.npy"
np.save(dataToStore, testR1000Err_30NP)
dataToStore = dataDir + "testR1000Err_60NP.npy"
np.save(dataToStore, testR1000Err_60NP)
dataToStore = dataDir + "testR1000Err_90NP.npy"
np.save(dataToStore, testR1000Err_90NP)
dataToStore = dataDir + "testR1000Err_120NP.npy"
np.save(dataToStore, testR1000Err_120NP)
dataToStore = dataDir + "testR1000Err_150NP.npy"
np.save(dataToStore, testR1000Err_150NP)
dataToStore = dataDir + "testR1000Err_180NP.npy"
np.save(dataToStore, testR1000Err_180NP)
dataToStore = dataDir + "testR1000Err_210NP.npy"
np.save(dataToStore, testR1000Err_210NP)
dataToStore = dataDir + "testR1000Err_240NP.npy"
np.save(dataToStore, testR1000Err_240NP)